In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns


In [31]:
file_paths = [
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/peugeot_207_02.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/peugeot_207_01.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/opel_corsa_01.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/opel_corsa_02.csv',
    ]

# Function to get the columns of a CSV file
def get_columns(file_path):
    return pd.read_csv(file_path, delimiter=';', encoding='latin1', nrows=0)

# Dictionary to hold file paths and their respective columns
file_columns = {file_path: get_columns(file_path).columns.tolist() for file_path in file_paths}

In [32]:
# Extract the columns of the first file to use as a reference
reference_columns = file_columns[file_paths[0]]

# Function to compare two lists
def lists_match(list1, list2):
    return list1 == list2

# Check if all files have the same structure
all_match = all(lists_match(reference_columns, cols) for cols in file_columns.values())

print(f"All files have the same structure: {all_match}")


All files have the same structure: True


In [33]:
# concatenate all CSV files into a single DataFrame
def concatenate_csv_files(file_paths):
    dfs = []  # List to store DataFrames
    for file_path in file_paths:
        df = pd.read_csv(file_path, delimiter=';', encoding='latin1')  # Load each file into a DataFrame
        dfs.append(df)  # Append the DataFrame to the list
    combined_df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
    return combined_df

# concatenate all your files
combined_df = concatenate_csv_files(file_paths)

#  inspect the combined DataFrame
print(combined_df.head())  # Print the first 5 rows of the combined dataframe

   Unnamed: 0  AltitudeVariation  VehicleSpeedInstantaneous  \
0          59          -0.599998                  24.299999   
1          60           0.099998                  27.900000   
2          61          -0.200001                  30.599998   
3          62          -0.899998                  29.699999   
4          63          -0.900002                  28.799999   

   VehicleSpeedAverage  VehicleSpeedVariance  VehicleSpeedVariation  \
0             8.720689             46.843772               6.299999   
1             9.045762             52.270798               3.600000   
2             9.405000             59.127937               2.699999   
3             9.900000             64.360674              -0.900000   
4            10.380000             68.519589              -0.900000   

   LongitudinalAcceleration  EngineLoad  EngineCoolantTemperature  \
0                   -1.2903   67.058823                      26.0   
1                   -1.4038    0.000000                 

This method helps identify columns with missing values and gives you an insight into the datatypes of each column.


In [41]:
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23775 entries, 0 to 23774
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 23775 non-null  int64  
 1   AltitudeVariation          23775 non-null  float64
 2   VehicleSpeedInstantaneous  23766 non-null  float64
 3   VehicleSpeedAverage        23775 non-null  float64
 4   VehicleSpeedVariance       23775 non-null  float64
 5   VehicleSpeedVariation      23775 non-null  float64
 6   LongitudinalAcceleration   23775 non-null  float64
 7   EngineLoad                 23770 non-null  float64
 8   EngineCoolantTemperature   23770 non-null  float64
 9   ManifoldAbsolutePressure   23770 non-null  float64
 10  EngineRPM                  23770 non-null  float64
 11  MassAirFlow                23770 non-null  float64
 12  IntakeAirTemperature       23770 non-null  float64
 13  VerticalAcceleration       23775 non-null  flo

The .describe() method shows a quick statistic summary of your numerical data.

In [39]:
combined_df.describe()


,Unnamed: 0,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage
count,23775.000000,23775.000000,23766.000000,23775.000000,23775.000000,23775.000000,23775.000000,23770.000000,23770.000000,23770.000000,23770.000000,23770.000000,23770.000000,23775.000000,23770.000000
mean,3410.669106,-0.263008,38.644538,38.811064,171.920888,-0.020554,0.681683,36.762356,70.818721,115.195120,1486.820257,16.045632,23.483004,0.463398,15.171096
std,2196.419996,2.091264,33.864438,31.149198,182.751192,2.475485,1.582911,26.640593,15.685020,20.455268,578.064539,9.692254,10.902487,3.462745,4.603774
min,44.000000,-24.600006,0.000000,0.000000,0.000000,-103.500000,-3.065000,0.000000,12.000000,88.000000,0.000000,0.880000,7.000000,-2.763100,7.271883
25%,1608.000000,-1.399996,11.700000,15.991532,46.048954,-0.900001,-0.219000,18.823530,63.000000,102.000000,854.000000,6.970000,16.000000,-0.554000,11.672288
50%,3150.000000,-0.099998,31.500000,29.314682,113.907731,0.000000,0.391000,31.372549,79.000000,107.000000,1485.000000,15.360000,19.000000,-0.078100,14.520122
75%,4906.500000,0.700001,54.899998,50.148162,230.200656,0.900002,1.159700,58.431374,81.000000,120.000000,1955.875000,21.717499,31.000000,0.237250,18.288337
max,8613.000000,11.400002,124.749725,121.330733,1418.370369,97.199997,8.477800,100.000000,89.000000,252.000000,3167.000000,73.250000,65.000000,17.944800,45.336861


In [44]:
print(combined_df['Unnamed: 0'].head())


0    59
1    60
2    61
3    62
4    63
Name: Unnamed: 0, dtype: int64


In [43]:
combined_df['drivingStyle'].value_counts()

drivingStyle
EvenPaceStyle      21016
AggressiveStyle     2759
Name: count, dtype: int64

drop rows with any missing values:

In [45]:
def check_first_index(file_path):
    df_temp = pd.read_csv(file_path, delimiter=';', encoding='latin1', nrows=1)
    first_index = df_temp.iloc[0]['Unnamed: 0']
    return first_index


In [46]:
file_paths = [
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/peugeot_207_02.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/peugeot_207_01.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/opel_corsa_01.csv',
    'C:/Users/daral/OneDrive - Atlantic TU/year 4/Gesture UI/project1/opel_corsa_02.csv',
]

# Check and print the first index of each file
for file_path in file_paths:
    first_index = check_first_index(file_path)
    print(f"First index in {file_path.split('/')[-1]}: {first_index}")


First index in peugeot_207_02.csv: 59
First index in peugeot_207_01.csv: 59
First index in opel_corsa_01.csv: 59
First index in opel_corsa_02.csv: 44


In [37]:
# df = df.dropna()


fill missing values with the median of the column:

In [38]:
# df = df.fillna(df.median())